In [0]:
import torch

In [2]:
from torchvision import datasets, transforms

training_data = datasets.FashionMNIST(".",download=True,train=True, transform=transforms.Compose([
                                                                                transforms.ToTensor(),
                                                                                lambda x: torch.flatten(x)
]))
test_data = datasets.FashionMNIST(".",download=True,train=False, transform=transforms.Compose([
                                                                                transforms.ToTensor(),
                                                                                lambda x: torch.flatten(x),
]))


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw
Processing...
Done!


In [3]:
type(training_data[1][0])

torch.Tensor

In [4]:
training_data[2][0].size()

torch.Size([784])

In [0]:
from torch import nn, optim

In [0]:
from tqdm import tqdm_notebook as tqdm
from torch.utils.data import RandomSampler
model_TD1 = nn.Sequential(nn.Linear(784, 1))

def trainer(dataset,model,loss_fn,epoch=10,batch_size=1,rate=1e-4):
  data = torch.utils.data.DataLoader(dataset, batch_size, RandomSampler)
  optimizer = optim.Adam(model.parameters(),lr=rate)
  for i in tqdm(range(epoch)):
    for imgs, labels in tqdm(data):
      imgs = imgs.cuda()
      labels = labels.cuda()
      optimizer.zero_grad()
      labels = labels.float()
      output = model(imgs)
      loss = loss_fn(output, labels)
      loss.backward()

      optimizer.step()

In [55]:
trainer(training_data, model_TD1.cuda(), nn.MSELoss(), batch_size=64, epoch=10)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [0]:
def success_rate(model,test_data):
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=len(test_data))
  imgs, labels = iter(test_loader).next()
  imgs = imgs.cuda()
  labels = labels.cuda()
  output = model(imgs)
  print(output)
  correct = [i for i,j in zip(output, labels) if torch.round(i) == j]
  print(len(correct))
  return(len(correct) / len(labels))

In [59]:
suc = success_rate(model_TD1, test_data)
suc

tensor([[3.6745],
        [5.3241],
        [4.9974],
        ...,
        [3.2376],
        [4.2832],
        [2.9110]], device='cuda:0', grad_fn=<AddmmBackward>)
696


0.0696

In [19]:
def trainer(dataset,model,loss_fn,epoch=10,batch_size=1,rate=1e-4):
  data = torch.utils.data.DataLoader(dataset, batch_size, RandomSampler)
  optimizer = optim.Adam(model.parameters(),lr=rate)
  for i in tqdm(range(epoch)):
    for imgs, labels in tqdm(data):
      imgs = imgs.cuda()
      labels = labels.cuda()

      optimizer.zero_grad()

      output = model(imgs)
      loss = loss_fn(output, labels)
      loss.backward()

      optimizer.step()


model_2 = nn.Sequential(nn.Linear(784, 10),
                        nn.LogSoftmax(1))
model_2.cuda()
trainer(training_data, model_2, nn.CrossEntropyLoss(), batch_size=64)

In [22]:
def success_rate(model,test_data):
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=len(test_data))
  imgs, labels = iter(test_loader).next()
  imgs = imgs.cuda()
  labels = labels.cuda()

  output = model(imgs)
  output = [torch.argmax(i) for i in output]
  correct = [i for i,j in zip(output, labels) if i == j]
  print(len(correct))
  return(len(correct) / len(labels))

success_rate(model_2, test_data)

8265


0.8265

In [37]:
model_3 = nn.Sequential(nn.Linear(784, 128),
                        nn.ReLU(),
                        nn.Linear(128,10),
                        nn.LogSoftmax(1))
model_3.cuda()
trainer(training_data, model_3, nn.CrossEntropyLoss(), batch_size=64)

In [38]:
success_rate(model_3, test_data)

8530


0.853

In [39]:
model_4 = nn.Sequential(nn.Linear(784, 128),
                        nn.Sigmoid(),
                        nn.Linear(128,10),
                        nn.LogSoftmax(1))
model_4.cuda()
trainer(training_data, model_4, nn.CrossEntropyLoss(), batch_size=64)
success_rate(model_4, test_data)

8396


0.8396

In [49]:
model_5 = nn.Sequential(nn.Linear(784, 128),
                        nn.Tanh(),
                        nn.Linear(128,10),
                        nn.LogSoftmax(1))
model_5.cuda()
trainer(training_data, model_5, nn.CrossEntropyLoss(), batch_size=8)
success_rate(model_5, test_data)

8737


0.8737

In [51]:
model_6 = nn.Sequential(nn.Linear(784, 128),
                        nn.Tanh(),
                        nn.Linear(128,10),
                        nn.LogSoftmax(1))
model_6.cuda()
trainer(training_data, model_6, nn.NLLLoss(), batch_size=8)
success_rate(model_6, test_data)

8732


0.8732

In [0]:
from google.colab import files
torch.save(model_5, 'model_5')